## Imports

In [19]:
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from open3d.visualization import gui
from open3d.visualization import rendering
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

### Load and compute the mean curvature

In [13]:
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh("stlFiles/sinifBirNumaraAlti.stl")

# Get vertices, faces, and normals
vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)

# Compute Mean Curvature using Trimesh
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, radius=2)
print('Discrete Mean curvature calculated')


Discrete Mean curvature calculated


### The function that extracts the cavity from the tooth model

In [14]:
def extract_largest_cavity(vertices, faces, cavity_indices):
    # Get unique cavity indices
    unique_cavity_indices = np.unique(cavity_indices)
    
    # Find faces that have all vertices in cavity_indices
    cavity_face_mask = np.isin(faces.ravel(), unique_cavity_indices).reshape(faces.shape)
    cavity_face_indices = np.where(np.all(cavity_face_mask, axis=1))[0]
    cavity_faces = faces[cavity_face_indices]
    
    # Create adjacency matrix for connected component analysis
    edges = set()
    for face in cavity_faces:
        edges.add((face[0], face[1]))
        edges.add((face[1], face[2]))
        edges.add((face[2], face[0]))
    
    # Create sparse adjacency matrix
    row, col = zip(*edges)
    row = np.array(row)
    col = np.array(col)
    data = np.ones_like(row)
    
    # Create sparse matrix with size equal to total vertices 
    # (will be pruned to cavity vertices later)
    adj_matrix = csr_matrix((data, (row, col)), shape=(vertices.shape[0], vertices.shape[0]))
    
    # Find connected components
    n_components, labels = connected_components(csgraph=adj_matrix, directed=False)
    
    # Count vertices in each component
    component_sizes = np.zeros(n_components, dtype=int)
    for label in labels[unique_cavity_indices]:
        component_sizes[label] += 1
    
    # Find the largest component
    largest_component = np.argmax(component_sizes)
    
    # Get vertices from the largest component
    largest_cavity_indices = np.where(labels == largest_component)[0]
    largest_cavity_indices = np.intersect1d(largest_cavity_indices, unique_cavity_indices)
    
    # Create index mapping for new mesh
    index_map = np.zeros(len(vertices), dtype=int)
    for i, idx in enumerate(largest_cavity_indices):
        index_map[idx] = i
    
    # Get faces for largest component
    largest_face_mask = np.isin(cavity_faces.ravel(), largest_cavity_indices).reshape(cavity_faces.shape)
    largest_face_indices = np.where(np.all(largest_face_mask, axis=1))[0]
    largest_cavity_faces = cavity_faces[largest_face_indices]
    
    # Remap face indices
    remapped_faces = np.zeros_like(largest_cavity_faces)
    for i in range(largest_cavity_faces.shape[0]):
        for j in range(3):
            remapped_faces[i, j] = index_map[largest_cavity_faces[i, j]]
    
    # Create and return the largest cavity mesh
    largest_cavity_mesh = o3d.geometry.TriangleMesh()
    largest_cavity_mesh.vertices = o3d.utility.Vector3dVector(vertices[largest_cavity_indices])
    largest_cavity_mesh.triangles = o3d.utility.Vector3iVector(remapped_faces)
    largest_cavity_mesh.compute_vertex_normals()
    
    # Set color for visualization
    cavity_colors = np.ones((len(largest_cavity_indices), 3)) * [0, 1, 0]  # Green
    largest_cavity_mesh.vertex_colors = o3d.utility.Vector3dVector(cavity_colors)
    
    return largest_cavity_mesh, largest_cavity_indices

### The function that extracts the cavity bottom 

In [4]:
def extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.1):
    # Get vertices and triangles from the mesh
    cavity_vertices = np.asarray(largest_cavity_mesh.vertices)
    cavity_triangles = np.asarray(largest_cavity_mesh.triangles)
    
    # Calculate z-range
    min_z = np.min(cavity_vertices[:, 2])
    max_z = np.max(cavity_vertices[:, 2])
    z_range = max_z - min_z
    
    # Define a threshold for what constitutes the "bottom"
    # Here we're considering the bottom 10% of the cavity's depth
    z_threshold = min_z + z_range * threshold_percentage
    
    # Find vertices that are in the bottom region
    bottom_vertex_mask = cavity_vertices[:, 2] <= z_threshold
    bottom_vertex_indices = np.where(bottom_vertex_mask)[0]
    
    # Find triangles where all three vertices are in the bottom region
    bottom_triangles_mask = np.isin(cavity_triangles.ravel(), bottom_vertex_indices).reshape(cavity_triangles.shape)
    bottom_triangle_indices = np.where(np.all(bottom_triangles_mask, axis=1))[0]
    
    if len(bottom_triangle_indices) == 0:
        print("No triangles found in the bottom region. Try adjusting the threshold.")
        return None
    
    # Create a new mesh for the bottom surface
    bottom_triangles = cavity_triangles[bottom_triangle_indices]
    
    # Get unique vertices used in the bottom triangles
    unique_vertices = np.unique(bottom_triangles.ravel())
    
    # Create index mapping
    index_map = np.zeros(len(cavity_vertices), dtype=int)
    for i, idx in enumerate(unique_vertices):
        index_map[idx] = i
    
    # Remap triangle indices
    remapped_triangles = np.zeros_like(bottom_triangles)
    for i in range(bottom_triangles.shape[0]):
        for j in range(3):
            remapped_triangles[i, j] = index_map[bottom_triangles[i, j]]
    
    # Create bottom surface mesh
    bottom_mesh = o3d.geometry.TriangleMesh()
    bottom_mesh.vertices = o3d.utility.Vector3dVector(cavity_vertices[unique_vertices])
    bottom_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)
    bottom_mesh.compute_vertex_normals()
    
    # Set color for visualization
    bottom_colors = np.ones((len(unique_vertices), 3)) * [0, 0, 1]  # Blue for bottom surface
    bottom_mesh.vertex_colors = o3d.utility.Vector3dVector(bottom_colors)
    
    return bottom_mesh

### The function that calulates the cavity_bottom roughness

In [5]:
def calculate_roughness(mesh):
    mesh_vertices = np.asarray(mesh.vertices)
    mesh_faces = np.asarray(mesh.triangles)
    tri_mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh_faces, process=False)
    normals = tri_mesh.face_normals
    adj = tri_mesh.face_adjacency

    # Compute angle between adjacent face normals
    dot = np.einsum('ij,ij->i', normals[adj[:, 0]], normals[adj[:, 1]])
    angles = np.arccos(np.clip(dot, -1.0, 1.0)) # Mean angle
    angles_deg = np.degrees(angles) # roughness
    return angles_deg


### kavitenin seçilmesi 

In [15]:
cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
largest_cavity_mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)
cavity_vertices= np.asarray(largest_cavity_mesh.vertices)


### Experimental : Get OBBs and visualize them

In [20]:
def get_rotation_matrix_from_two_vectors(v1, v2):
    """
    Return a rotation matrix that rotates vector v1 to align with vector v2.
    """
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)

    cross = np.cross(v1, v2)
    dot = np.dot(v1, v2)

    if np.isclose(dot, -1.0):
        # Opposite vectors – rotate 180 degrees around any perpendicular axis
        orthogonal = np.array([1, 0, 0]) if not np.allclose(v1, [1, 0, 0]) else np.array([0, 1, 0])
        axis = np.cross(v1, orthogonal)
        axis /= np.linalg.norm(axis)
        angle = np.pi
    elif np.isclose(dot, 1.0):
        return np.eye(3)
    else:
        axis = cross / np.linalg.norm(cross)
        angle = np.arccos(dot)

    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ])

    R = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)
    return R


In [28]:
# Convert full tooth to Open3D mesh
tooth_o3d = o3d.geometry.TriangleMesh()
tooth_o3d.vertices = o3d.utility.Vector3dVector(vertices)
tooth_o3d.triangles = o3d.utility.Vector3iVector(faces)
tooth_o3d.compute_vertex_normals()
tooth_o3d.paint_uniform_color([0.8, 0.8, 0.8])  # light gray

# ---------------------------------------------------
# 🟩 Compute OBBs
# ---------------------------------------------------

tooth_obb = tooth_o3d.get_oriented_bounding_box()
tooth_obb.color = (0.0, 0.0, 1.0)  # blue

cavity_obb = largest_cavity_mesh.get_oriented_bounding_box()
cavity_obb.color = (1.0, 0.0, 0.0)  # red

# ---------------------------------------------------
# 🧮 Get dimensions
# ---------------------------------------------------

def get_obb_dims(obb):
    extent = obb.extent  # [width, height, depth] along principal axes
    R = obb.R  # rotation matrix
    axes = R.T  # columns of R are direction vectors
    return extent, axes, obb.center

tooth_extent, tooth_axes, tooth_center = get_obb_dims(tooth_obb)
cavity_extent, cavity_axes, cavity_center = get_obb_dims(cavity_obb)

print("\n📏 Tooth OBB Dimensions (X,Y,Z):", tooth_extent)
print("📏 Cavity OBB Dimensions (X,Y,Z):", cavity_extent)

# ---------------------------------------------------
# 🧭 Visualize Arrows for Length / Height / Depth
# ---------------------------------------------------

def create_arrow(center, axis, length, color):
    arrow = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.2,
        cone_radius=0.4,
        cylinder_height=length - 1.0,
        cone_height=1.0,
        resolution=20,
        cylinder_split=4,
        cone_split=1
    )
    arrow.paint_uniform_color(color)

    # Align the arrow with the axis
    default_dir = np.array([0, 1, 0])  # default arrow points in +Y
    axis = axis / np.linalg.norm(axis)
    rot_matrix = get_rotation_matrix_from_two_vectors(default_dir, axis)
    arrow.rotate(rot_matrix, center=(0, 0, 0))

    # Translate to center
    arrow.translate(center)
    return arrow

# Arrows for cavity
cavity_arrows = []
colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # X (red), Y (green), Z (blue)
for i in range(3):
    arrow = create_arrow(cavity_center, cavity_axes[i], cavity_extent[i], colors[i])
    cavity_arrows.append(arrow)

# Arrows for tooth
tooth_arrows = []
for i in range(3):
    arrow = create_arrow(tooth_center, tooth_axes[i], tooth_extent[i], colors[i])
    tooth_arrows.append(arrow)
    
points = np.asarray(tooth_o3d.vertices)

# Run PCA to estimate axes of the tooth
pca = PCA(n_components=3)
pca.fit(points)
axes = pca.components_  # Each row is a principal axis
center = pca.mean_

# Project all points to each axis
projected = points - center
projections = projected @ axes.T  # shape (N, 3)

# Compute distances (peak-to-peak range along each axis)
ranges = projections.ptp(axis=0)

# Output approximate anatomical axis matches
print("\n🦷 Estimated Axis Directions (rows = principal axes):\n", axes)
print("\n📏 Tooth Dimensions along PCA axes (in mm):")
print("  Axis 1 (Likely Buccolingual):", ranges[0])
print("  Axis 2 (Likely Mesiolingual or Mesiodistal):", ranges[1])
print("  Axis 3 (Likely Occluso-apical or height):", ranges[2])


# ---------------------------------------------------
# 🧿 Final Visualization
# ---------------------------------------------------
def create_axis_lines(center, axes, lengths, colors):
    lines = []
    for i in range(3):
        p1 = center - axes[i] * lengths[i] / 2
        p2 = center + axes[i] * lengths[i] / 2
        line = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector([p1, p2]),
            lines=o3d.utility.Vector2iVector([[0, 1]])
        )
        line.colors = o3d.utility.Vector3dVector([colors[i]])
        lines.append(line)
    return lines

# Choose lengths based on PCA spread
colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # X = red, Y = green, Z = blue
axis_lines = create_axis_lines(center, axes, ranges, colors)


o3d.visualization.draw_geometries(
    [
        tooth_o3d,
        largest_cavity_mesh,
        tooth_obb,
        cavity_obb,
        *axis_lines
        # *tooth_arrows,
        # *cavity_arrows
    ],
    zoom=0.7,
    mesh_show_back_face=True,
    window_name="Tooth + Cavity OBBs"
)



📏 Tooth OBB Dimensions (X,Y,Z): [12.21253385 11.92310044  8.93872114]
📏 Cavity OBB Dimensions (X,Y,Z): [8.46000958 3.66252435 2.58655567]

🦷 Estimated Axis Directions (rows = principal axes):
 [[ 0.99423497  0.10329265  0.02876544]
 [-0.1036276   0.99456176  0.01040337]
 [-0.02753441 -0.01332429  0.99953205]]

📏 Tooth Dimensions along PCA axes (in mm):
  Axis 1 (Likely Buccolingual): 11.885112589965956
  Axis 2 (Likely Mesiolingual or Mesiodistal): 11.052254831991833
  Axis 3 (Likely Occluso-apical or height): 8.334168612283754


### Kavitenin alt kısmının seçilmesi ve kavite yüksekliğinin hesaplanması

In [29]:
cavity_bottom = extract_cavity_bottom(largest_cavity_mesh, threshold_percentage=0.4)

# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)
bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

# kavite alanının en üstü 
max_z = np.max(cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)



### Roughness Hesaplanması

In [30]:

outline_indices = np.where((mean_curvature > 3.0))[0]
roughness = calculate_roughness(cavity_bottom)
print("Mean angle between adjacent faces:", np.mean(roughness))
print("Standard deviation (roughness):", np.std(roughness))

Mean angle between adjacent faces: 6.076013062789848
Standard deviation (roughness): 4.213117516385517


### Görüntüleme

In [31]:
# **Çizgiyi tanımlama**
cavity_centroid = np.mean(cavity_vertices, axis=0)
min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector([min_z_point, max_z_point])
line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0]])  # Mavi çizgi

print("cavity_depth : ",cavity_depth)
print("roughness : ",roughness)


o3d.visualization.draw_geometries([line_set,cavity_bottom])

cavity_depth :  2.0301277332356844
roughness :  [18.09196629  3.21604171  7.5380052  ...  9.02044369  0.92320791
 10.34273216]
